In [2]:
# Linear Regression project with generalized input dimensions and L2 normalisation
__author__ = "Billy Cao"
import tensorflow as tf
from math import sqrt
import generator

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)  # change to false when running on ML server
tf.keras.mixed_precision.set_global_policy('mixed_float16')

batchSize = 100
gen = generator.gen2d(batchSize)  # gen2d for 2D input
epochs = 500
regTerm = 0.001
learning_rate = 1e-5

class LinearModel:  # initializing to 1 now, but also can do 0 or random
    def __call__(self, x):  # predicting function
        return self.Weight * x + self.Bias

    def __init__(self):
        self.Weight = tf.Variable(1.0, shape=tf.TensorShape(None))  # initialize m to any shape
        self.Bias = tf.Variable(1.0)


def loss(y, pred):  # Mean Squared Error with L2 Normalisation
    return tf.reduce_mean(tf.square(y - pred)) + tf.reduce_sum(regTerm * tf.square(linear_model.Weight))


def train(linear_model, x, y, lr):
    # use to reshape into 2D vector if input is of unknown dimension
    # if len(x.shape) == 1:
    #     X = tf.reshape(x, [x.shape[0], 1])
    with tf.GradientTape(persistent=False) as t:  # persistent=True is needed if assigning dy_dWeight, dy_dBias in 2 lines. Limits the times u can call it to once
        current_loss = loss(y, linear_model(x))
    dy_dWeight, dy_dBias = t.gradient(current_loss, [linear_model.Weight, linear_model.Bias])
    linear_model.Weight.assign_sub(lr * dy_dWeight)
    linear_model.Bias.assign_sub(lr * dy_dBias)


linear_model = LinearModel()
sampleX, sampleY = next(gen)
linear_model.Weight.assign([1.0] * sampleX.shape[-1])  # initialize m to 1.0 and make it same dimension as the input
for epoch_count in range(epochs):
    x, y = next(gen)
    real_loss = loss(y, linear_model(x))
    train(linear_model, x, y, lr=learning_rate)
    print(f"Epoch count {epoch_count}: Loss: {real_loss.numpy()}")

# Assuming the weights are correct, the loss will be directly related to error in Bias. Thus we compensate it here
linear_model.Bias.assign_sub(linear_model.Bias - sqrt(real_loss.numpy()))
print(f'Weight: {linear_model.Weight.numpy()}, Bias: {linear_model.Bias.numpy()}')

Epoch count 0: Loss: 2070.170654296875
Epoch count 1: Loss: 1812.0076904296875
Epoch count 2: Loss: 1965.1500244140625
Epoch count 3: Loss: 2067.736328125
Epoch count 4: Loss: 1897.821533203125
Epoch count 5: Loss: 1484.716064453125
Epoch count 6: Loss: 1355.83056640625
Epoch count 7: Loss: 1456.6849365234375
Epoch count 8: Loss: 1446.198486328125
Epoch count 9: Loss: 1382.376953125
Epoch count 10: Loss: 1125.7061767578125
Epoch count 11: Loss: 1438.1036376953125
Epoch count 12: Loss: 1140.7099609375
Epoch count 13: Loss: 1236.64306640625
Epoch count 14: Loss: 1406.0333251953125
Epoch count 15: Loss: 1114.1751708984375
Epoch count 16: Loss: 1035.7381591796875
Epoch count 17: Loss: 927.2775268554688
Epoch count 18: Loss: 991.0014038085938
Epoch count 19: Loss: 814.9093627929688
Epoch count 20: Loss: 985.4908447265625
Epoch count 21: Loss: 1012.75341796875
Epoch count 22: Loss: 831.1987915039062
Epoch count 23: Loss: 732.0519409179688
Epoch count 24: Loss: 821.6946411132812
Epoch count 2